# Phase segregation of a block polymer

### Import necessary modules

In [ ]:
import sys
sys.path.append('../polysim')
import builder
import vmd
import os
import lammps_log
from glob import glob
from matplotlib import pyplot
from PIL import Image
from IPython.display import display, Video
import subprocess

### Builds initial MD system (diblock polymer model)

In [ ]:
n = 200
m = 100
system = builder.construct_random_chains(molecules=n, molecule_size=m)

# Change half the atoms in each change to type 2.
for i in range(n):
    for j in range(m):
        if j > m // 2:
            system.atom_type[j+i*m] = 2

system.masses.append(1.0)
system.write_lammps_data('polymer.lammps')

### Runs MD simulation (should take about 3 minutes)

In [ ]:
! mpiexec -n ${SLURM_CPUS_PER_TASK} --oversubscribe lmp_mpi -in in.lammps

### VMD script for animating results (takes about 3 minutes)

In [ ]:
vmd_cmd = ['vmd', '-dispdev', 'none']
vmd_script = '''
mol new {dump.lammpstrj} type {lammpstrj} first 0 last -1 step 1 waitfor all autobonds off
display projection Orthographic
display shadows on
display ambientocclusion on
display depthcue off
color Display Background white
axes location Off

mol modselect 0 0 type 2
mol modstyle 0 0 VDW 0.2 12
mol modcolor 0 0 Type
mol modmaterial 0 0 AOChalkyvmd 
mol addrep 0
mol modselect 1 0 type 1
mol modstyle 1 0 VDW 0.2 12
mol modcolor 1 0 Type
mol modmaterial 1 0 AOChalky

color Type 1 lime
color Type 2 iceblue
display update
rotate x by 30
rotate y by 30
set numframes [molinfo 0 get numframes]
for {set i 0} {$i < $numframes} {incr i} {
animate goto $i
set filename frames/frame[format "%04d" $i].tga
render TachyonInternal $filename
}
quit '''

In [ ]:
if not os.path.exists('frames'):
    os.mkdir('frames')
out = subprocess.run(vmd_cmd, input=vmd_script, text=1, env=os.environ)
for f in glob('frames/frame*.tga'):
    cmd = ['convert', f, f.replace('tga', 'jpg')]
    subprocess.run(cmd, capture_output=1, text=1)

In [ ]:
! ffmpeg -y -r 6 -f image2 -i frames/frame%04d.jpg -vf "scale=640:-2" -vcodec libx264 -pix_fmt yuv420p -crf 30 animation.mp4 &> /dev/null
Video('animation.mp4')